In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('Dataset1.csv')
criteria=pd.read_csv('criteria.csv')

In [3]:
data

,name,exp,tot bugs asgn,new bugs,tot fixed bugs,avg fixing time(days)
0,Adam Lock,3.29,21,3,18,408
1,Alec Flett,3.90,23,10,13,346
2,Ben Campbell,3.65,12,2,10,354
3,Charles Manske,3.66,24,9,15,514
4,Ginn Chen,3.26,21,5,16,669
5,Julian Reschke,3.40,19,6,13,219
6,Kestrel,3.41,19,7,12,158
7,Michael Ventnor,3.46,17,3,14,254
8,Yulia Startsev,3.29,26,13,13,179
9,Ryan Cassin,3.15,17,4,13,351


In [4]:
criteria


,Weights,Impacts
0,0.30,+
1,0.15,+
2,0.15,+
3,0.35,+
4,0.15,-


In [5]:
import pandas as pd
import numpy as np

def topsis(df,dp):
    weights = dp['Weights'].values
    impacts = dp['Impacts'].values
#     df = df.drop(columns=['Weights','Impacts'])
    #Normalize the matrix
    normalized_matrix = df.copy()
    for column in normalized_matrix.columns:
        normalized_matrix[column] = normalized_matrix[column] / np.sqrt(np.sum(normalized_matrix[column] ** 2))

    #Multiply the normalized matrix by the weights
    weighted_matrix = normalized_matrix.copy()
    for i, weight in enumerate(weights):
        weighted_matrix.iloc[:, i] = weighted_matrix.iloc[:, i] * weight
        

    #Determine the ideal and negative-ideal solutions
    ideal_best = np.zeros(len(weighted_matrix.columns))
    ideal_worst = np.zeros(len(weighted_matrix.columns))
    for i in range(len(weighted_matrix.columns)):
        if impacts[i] == '+':
            ideal_best[i] = np.max(weighted_matrix.iloc[:, i])
            ideal_worst[i] = np.min(weighted_matrix.iloc[:, i])
        elif impacts[i] == '-':
            ideal_best[i] = np.min(weighted_matrix.iloc[:, i])
            ideal_worst[i] = np.max(weighted_matrix.iloc[:, i])
       
    #Calculate the Euclidean distances to the ideal and negative-ideal solutions
    s_best = np.sqrt(np.sum((weighted_matrix - ideal_best) ** 2, axis=1))
    s_worst = np.sqrt(np.sum((weighted_matrix - ideal_worst) ** 2, axis=1))

    #Calculate the relative closeness to the ideal solution
    performance_score = s_worst / (s_best + s_worst)

    #Rank alternatives based on performance score
    df['Performance Score'] = performance_score
    df['Rank'] = df['Performance Score'].rank(ascending=False)
    return df

if __name__ == "__main__":
    # Read data from CSV
    data = pd.read_csv('Dataset1.csv')
    data1=pd.read_csv('criteria.csv')
    #weghts and impacts
    
    result = topsis(data.iloc[:, 1:],data1.iloc[:, :])
    print(result)

    exp  tot bugs asgn  new bugs  tot fixed bugs  avg fixing time(days)  \
0  3.29             21         3              18                    408   
1  3.90             23        10              13                    346   
2  3.65             12         2              10                    354   
3  3.66             24         9              15                    514   
4  3.26             21         5              16                    669   
5  3.40             19         6              13                    219   
6  3.41             19         7              12                    158   
7  3.46             17         3              14                    254   
8  3.29             26        13              13                    179   
9  3.15             17         4              13                    351   

   Performance Score  Rank  
0           0.494361   6.0  
1           0.606617   2.0  
2           0.283470  10.0  
3           0.553188   3.0  
4           0.389645   8.0  
